In [ ]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

In [ ]:
gds.run_cypher("""CREATE INDEX point_row_col IF NOT EXISTS
FOR (p:Point) ON (p.row, p.col)""")

In [ ]:
input_file = "input.txt"

In [ ]:
lines = len(open(input_file, 'r').readlines())
width = len(open(input_file, 'r').readline()) - 1

gds.run_cypher("""
UNWIND RANGE (0, 2*($lines-1)) AS row
UNWIND RANGE (0, 2*($width-1)) AS col
CALL {
WITH row, col
MERGE (:Point {row:row/2.0, col:col/2.0})
} IN TRANSACTIONS OF 10000 ROWS
""", {'lines':lines, 'width':width})


In [ ]:
gds.run_cypher("""
MATCH (p:Point)
WITH p.row AS row, p.col AS col, p
ORDER BY col
WITH row, collect(p) AS point_row
CALL {
    WITH point_row
    CALL apoc.nodes.link(point_row,"ADJACENT")
} IN TRANSACTIONS OF 10 ROWS
""")

In [ ]:
gds.run_cypher("""
MATCH (p:Point)
WITH p.row AS row, p.col AS col, p
ORDER BY row
WITH col, collect(p) AS point_col
CALL {
    WITH point_col
    CALL apoc.nodes.link(point_col,"ADJACENT")
} IN TRANSACTIONS OF 10 ROWS
""")

In [ ]:

file = open(input_file, 'r')
values = []
for row, line in enumerate(file):
    query ="""
    WITH $line AS line, $row AS row
    WITH split(line, '')[0..-1] AS line, row
    WITH [ix IN range(0, size(line)-1) | [ix, line[ix]]] AS line, row
    UNWIND line AS col_sym
    MATCH (p:Point {row:row, col:col_sym[0]})
    SET p:Tile, p.sym=col_sym[1]
    """
    gds.run_cypher(query, {"row":row, "line":line})

In [ ]:
gds.run_cypher("""MATCH (ref:Point:Tile)
CALL{ WITH ref
MATCH (east:Point:Tile WHERE east.row = ref.row and east.col = ref.col + 1)
MERGE (ref)-[:HAS_NEIGHBOR {dir:'E'}]->(east)
MERGE (ref)<-[:HAS_NEIGHBOR {dir:'W'}]-(east)
} IN TRANSACTIONS OF 1000 ROWS""")

In [ ]:
gds.run_cypher("""MATCH (ref:Point:Tile)
CALL{ WITH ref
MATCH (south:Point:Tile WHERE south.col = ref.col and south.row = ref.row + 1)
MERGE (ref)-[:HAS_NEIGHBOR {dir:'S'}]->(south)
MERGE (ref)<-[:HAS_NEIGHBOR {dir:'N'}]-(south)
} IN TRANSACTIONS OF 1000 ROWS""")

In [ ]:
connectable = {'|': ['N', 'S'],
'-': ['E', 'W'],
'L': ['N', 'E'],
'J': ['N', 'W'],
'7': ['S', 'W'],
'F': ['S', 'E'],
'.': [],
'S': ['N', 'S', 'E', 'W']}

In [ ]:
gds.run_cypher("""
WITH $connectable AS connectable
MATCH (p:Tile)-[r:HAS_NEIGHBOR WHERE NOT r.dir IN connectable[p.sym]]->()
DELETE r
""", {'connectable':connectable})

In [ ]:
gds.run_cypher("""MATCH (a:Point)-[r:HAS_NEIGHBOR]->(b:Point)
WHERE NOT EXISTS {(b)-[:HAS_NEIGHBOR]->(a)}
SET r.toDelete = TRUE""");
gds.run_cypher("MATCH ()-[r {toDelete:TRUE}]->() DELETE r");

In [ ]:
gds.run_cypher("""
MATCH (source:Point)-[r:HAS_NEIGHBOR]->(target:Point)
WITH gds.graph.project('giant_cycle', source, target) AS g
RETURN
  g.graphName AS graph, g.nodeCount AS nodes, g.relationshipCount AS rels
""")

In [ ]:
gds.run_cypher("""
MATCH (source:Point {sym: 'S'})
CALL gds.allShortestPaths.dijkstra.stream('giant_cycle', {
    sourceNode: source
})
YIELD totalCost
RETURN toInteger(max(totalCost)) AS part1
""")

# Part 2

In [ ]:
Ggr = gds.graph.get("giant_cycle")
gds.wcc.write(Ggr, writeProperty="pipe_cc")

In [ ]:
gds.run_cypher("""
MATCH (source:Tile {sym: 'S'})
MATCH (tile:Tile {pipe_cc: source.pipe_cc})
SET tile:Border
""")

In [ ]:
gds.run_cypher("""
MATCH (a:Tile:Border)-[:ADJACENT]->(fp)-[:ADJACENT]->(b:Tile:Border)
WHERE EXISTS {(a)-[:HAS_NEIGHBOR]-(b)}
CALL {
WITH fp
SET fp:Border
} IN TRANSACTIONS OF 1000 ROWS
""")

In [ ]:
gds.run_cypher("""
MATCH (source:Point&!Border)
OPTIONAL MATCH (source)-[r:ADJACENT]->(target:Point&!Border)
WITH gds.graph.project('regions', source, target) AS g
RETURN
  g.graphName AS graph, g.nodeCount AS nodes, g.relationshipCount AS rels
""")

In [ ]:
G = gds.graph.get("regions")

In [ ]:
gds.wcc.write(G, writeProperty="region")

In [ ]:
gds.run_cypher("""
MATCH (p:Point)
WITH p, p.col AS col, p.row AS row
WITH [min(col),max(col)] AS ext_col, [min(row),max(row)] AS ext_row
MATCH (p:Point)
WHERE p.row IN ext_row OR p.col IN ext_col
SET p:ExteriorOrBorder
""")

In [ ]:
gds.run_cypher("""
MATCH (inter:Tile&!Border)
WHERE NOT EXISTS {(ext:Point&ExteriorOrBorder {region:inter.region})}
SET inter:Interior
RETURN count(inter) AS part2
""")